In [1]:
from policyengine.countries import US
from policyengine.utils.charts import plotly_json_to_fig
from blank_slate_ubi_us.common import blank_slate_ubi
from ubicenter import format_fig

us = US()

blank_slate = blank_slate_ubi()

baseline, reformed = us._get_microsimulations(blank_slate)

charts = us.population_reform(blank_slate)

## Figure 1

In [2]:
format_fig(plotly_json_to_fig(charts["intra_income_decile_chart"]).update_layout(
    title="Distribution of gains and losses by income decile under Blank Slate UBI",
))

## Figure 2

In [3]:
from blank_slate_ubi_us.charts.age_winners import age_winner_chart

format_fig(plotly_json_to_fig(age_winner_chart(baseline, reformed, us.results_config)).update_layout(
    title="Distribution of gains and losses by age under Blank Slate UBI",
))

## Figure 3

In [4]:
from blank_slate_ubi_us.charts.state_choropleth import us_state_choropleth

us_state_choropleth(baseline, reformed)

## Figure 4

In [5]:
format_fig(plotly_json_to_fig(charts["poverty_chart"]).update_layout(
    title="Change to poverty rates by age group under Blank Slate UBI",
))

In [6]:
format_fig(plotly_json_to_fig(charts["rel_income_decile_chart"]).update_layout(
    title="Change to net income by income decile under Blank Slate UBI",
))

In [7]:
format_fig(plotly_json_to_fig(charts["inequality_chart"]).update_layout(
    title="Changes to inequality metrics under Blank Slate UBI",
))

In [8]:
from blank_slate_ubi_us.charts.state_poverty_choropleth import us_state_poverty_choropleth

us_state_poverty_choropleth(baseline, reformed)

In [9]:
from blank_slate_ubi_us.charts.program_winners import program_winner_chart

program_winner_chart(baseline, reformed, us.results_config)